In [4]:
import os

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from pykrige.ok import OrdinaryKriging

from air_brain.data.get_data import DATA_DIR
from air_brain.util.air import PM25
from air_brain.util.od import od

bg_file = os.path.join(DATA_DIR, "tl_2010_42003_bg10", "tl_2010_42003_bg10.shp")
zip_file = os.path.join(DATA_DIR, "zipcodes.geojson")

In [38]:
bg = gpd.read_file(bg_file)[["GEOID10", "geometry"]].rename(columns={"GEOID10": "ID"})
bg.ID = bg.ID.astype(int)
bg = bg.to_crs("EPSG:2272")
bg["bg_area"] = bg.area
bg.head()

,ID,geometry,bg_area
0,420035003001,"POLYGON ((1377682.744 377136.114, 1377722.374 ...",1.176803e+07
1,420034994002,"POLYGON ((1372769.868 367062.296, 1372766.616 ...",2.181285e+06
2,420034994001,"POLYGON ((1372986.574 363752.319, 1372642.405 ...",1.822751e+07
3,420034993002,"POLYGON ((1373979.039 371060.403, 1374390.811 ...",1.596451e+07
4,420034980002,"POLYGON ((1379500.042 364821.829, 1379475.641 ...",2.647614e+07


In [49]:
# to zipcode
zc = gpd.read_file(zip_file)[["ZIP", "geometry"]]
zc = zc.to_crs("EPSG:2272")
zc["zc_area"] = zc.area

df = gpd.overlay(zc, bg, how="intersection")
df["int_area"] = df["geometry"].area
df[["ZIP", "ID", "int_area"]]

,ZIP,ID,int_area
0,15007,420034080013,9.591285e+02
1,15007,420034080012,5.766900e+03
2,15007,420034080011,1.336382e+07
3,15007,420034070014,6.694539e+02
4,15014,420034035004,7.750012e+03
...,...,...,...
2157,15126,420035640001,6.573323e+07
2158,15126,420034530031,1.169512e+02
2159,15026,420034520001,1.320185e+08
2160,15026,420034520004,1.579788e+07


In [45]:
# check that total areas are close...
# ... why are they not close
grped = df.groupby("ZIP").agg({'zc_area': 'max',
                               'bg_intersect_area': 'sum'}).reset_index()
grped["area_diff"] = grped.zc_area - grped.bg_intersect_area
grped["diff_frac"] = grped.area_diff / grped.zc_area
grped

,ZIP,zc_area,bg_intersect_area,area_diff,diff_frac
0,15003,1.156317e+07,1.136988e+07,1.932971e+05,1.671661e-02
1,15005,1.084622e+08,1.071862e+08,1.276014e+06,1.176459e-02
2,15006,3.706483e+06,3.706483e+06,2.048910e-08,5.527908e-15
3,15007,1.337122e+07,1.337122e+07,-1.043081e-07,-7.800945e-15
4,15012,3.551101e+06,3.551101e+06,1.210719e-08,3.409420e-15
...,...,...,...,...,...
120,16046,9.537777e+07,9.527392e+07,1.038504e+05,1.088832e-03
121,16056,4.610618e+06,3.686766e+06,9.238523e+05,2.003749e-01
122,16059,2.296081e+07,2.291048e+07,5.033486e+04,2.192207e-03
123,16066,4.187091e+04,3.982111e+04,2.049796e+03,4.895514e-02


In [48]:
# example for later
ej = pd.read_csv(os.path.join(DATA_DIR, "epa_ej", "2015.csv"))[["ID", "PM25"]]
ej = ej.merge(df, on="ID", how="outer", validate="1:m")
ej["pm25_x_area"] = ej.PM25 * ej.bg_intersect_area
grped_ej = ej.groupby("ZIP").agg({"pm25_x_area": "sum",
                                  "bg_intersect_area": "sum"}).reset_index()
grped_ej["PM25"] = grped_ej.pm25_x_area / grped_ej.bg_intersect_area
grped_ej

,ZIP,pm25_x_area,bg_intersect_area,PM25
0,15003,1.346790e+08,1.136988e+07,11.845248
1,15005,1.257431e+09,1.071862e+08,11.731273
2,15006,4.284974e+07,3.706483e+06,11.560754
3,15007,1.550718e+08,1.337122e+07,11.597430
4,15012,4.202436e+07,3.551101e+06,11.834178
...,...,...,...,...
120,16046,1.113990e+09,9.527392e+07,11.692494
121,16056,4.241721e+07,3.686766e+06,11.505263
122,16059,2.657815e+08,2.291048e+07,11.600871
123,16066,4.671519e+05,3.982111e+04,11.731263
